In [1]:
import os
import json
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import Word2Vec

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

In [3]:
from itertools import chain
from collections import defaultdict

# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

# tag 중에 빈도수가 1번 이상 나오고, 플레이리스트 당 tag가 1개 이상인 것들만 포함
tags_more_than_one = [[tag for tag in tags if tags_frequency[tag] > 1 ] for tags in tags_all if len(tags) > 1]

In [4]:
# 전처리 후 태그들의 갯수
len(tags_more_than_one)

87829

In [5]:
# 전처리한 tag들을 가지고 gensim corpora 딕셔너리 만들기
tag_unique_dict = corpora.Dictionary(tags_more_than_one)

In [6]:
# gensim 딕셔너리의 token2id 함수를 이용해서 {태그:id} 딕셔너리 만들기
tags_to_id = tag_unique_dict.token2id

In [7]:
# 위 딕셔너리 key, value 반전해서 {id:태그} 딕셔너리 만들기
id_to_tags = {v:k for k,v in tags_to_id.items()}

In [8]:
# 벡터 크기 100으로 해서 window=4 주고 Word2Vec 학습
w2v_model = Word2Vec(sentences=tags_more_than_one,vector_size=100,window=4,min_count=1,workers=4,sg=1)

In [9]:
# w2v 모델 저장
w2v_model.save('./models/w2v_model')

In [10]:
# w2v 모델 불러오기
w2v_model = Word2Vec.load('./models/w2v_model')

In [11]:
# word2vec {태그:id} 딕셔너리 만들기
tag_to_index = w2v_model.wv.key_to_index

In [12]:
# word2vec key만 뽑기
index_to_key = w2v_model.wv.index_to_key

In [13]:
def mean_multiple_tags(model,tags,vector_length):
    
    sum_vector = np.zeros(vector_length)
    for tag in tags:
        sum_vector = sum_vector + model.wv[tag]
    mean_vector = sum_vector / len(tags)
    w2v_model.wv.add_vectors('new_vector', mean_vector)
    w2v_model.wv.fill_norms('new_vector')
    
    return w2v_model.wv.most_similar('new_vector')

In [16]:
mean_multiple_tags(w2v_model,['힙합','사랑','추억','이별'],w2v_model.wv.vector_size)

[('감미로움', 0.8901172876358032),
 ('쓸쓸하게', 0.8873785734176636),
 ('회상하며', 0.8858362436294556),
 ('붸붸', 0.8830861449241638),
 ('보이스', 0.8813999891281128),
 ('추억을', 0.880141019821167),
 ('추억에', 0.8760725259780884),
 ('소주한잔', 0.8714550733566284),
 ('기억', 0.8705544471740723),
 ('갬성갬성', 0.8696315288543701)]